# This notebook is used to scrape the top rental movies on Cineplex website.

In [0]:
# import custom module
from customlib.config.configdirectory import config_directory
from customlib.custom_logger.customlogger import set_logger
from customlib.custom_selenium.init_chrome import init_chrome_browser
from customlib.custom_selenium.lastest_download import latest_download_file
from customlib.custom_cineplex.RetrieveRental import RetrieveRental
from customlib.custom_cineplex.save_rentals_to_csv import save_rentals_to_csv_file

# import selenium module
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# import some basic modules
import os
import time
import csv
from datetime import datetime

# initilize config variable
my_config = config_directory('cineplex.ini','directory')
blob_config = config_directory('blobstorage.ini','blob_storage')

storage_name = blob_config["storage_name"]
storage_key = blob_config["key"]
container_name = blob_config["container_name"]
log_file_name = "cineplex_scraper"
inside_log_name = "cineplex_scraper"

# set logger
logger, log_dir, log_name = set_logger(log_file_name)
            
# download url
url = my_config['url']

# path to the chrome driver
chrome_path = my_config['chrome_path']

# in this case I don't need download path for the init_chrome_browser() method but I still put a directory anyway
download_path = "/mnt/container-data/"

# set chrome driver
driver = init_chrome_browser(download_path, chrome_path, url)

# set timeout
timeout = 5
try:
    # path to all the movie list
    xpath_movie_list = my_config['xpath_movie_list']
    # wait until all the list links visible
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, xpath_movie_list)))
except TimeoutException as err:
    print("Timed out waiting for page to load")
    logger.error("An error message: {}".format(err))

# get all the top rental links using list comprehension combined with selenium
class_link = "movie-pdp-link"
titles_links = [item.get_attribute("href") for item in WebDriverWait(driver, timeout). \
                   until(EC.presence_of_all_elements_located((By.CLASS_NAME, class_link)))]

# create empty list    
rentals = []
i = 1

# append data into the list
for link in titles_links:
    driver.get(link)
    print(f'{datetime.now()}    downloading #{i}: {link} .......')
    i += 1 
    try:
        # xpath string linked to "title", "year", "synopsis" data in the html
        xpath_title_1    = my_config['xpath_title_1']
        xpath_title_2    = my_config['xpath_title_2']
        xpath_year       = my_config['xpath_year']
        xpath_synopsis_1 = my_config['xpath_synopsis_1']
        xpath_synopsis_2 = my_config['xpath_synopsis_2']

        # retrieve data
        title = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_title_1))).text
        year  = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_year))).text
        synopsis = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_synopsis_1))).text 
       
        # slow down the execution
        time.sleep(5)

    except TimeoutException as err:
        # retrieve data due to different html setup
        title = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_title_2))).text
        synopsis = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, xpath_synopsis_2))).text 
        logger.warning("An info message: {}".format(err))
        
    except Exception as err:
        logger.error("An error message: {}".format(err))
        
    # append data to rentals list
    try:
        rentals.append(RetrieveRental(title, year, synopsis))
        
    except Exception as err:
        logger.error("An error message: {}".format(err))

# write csv file
try: 
    directory_1 = my_config['directory_save1'] + datetime.today().strftime('%Y-%m-%d')+ ".csv"
    logger.info("Successfully scrapped the top 36 rental movies")   
except Exception as err:
    logger.error("An error message: {}".format(err))

# save csv file to the directory
try:
    save_rentals_to_csv_file(rentals_list = rentals, file_name = directory_1)
    logger.info("Successfully saved cineplex data to DBFS storage")
except Exception as err:
    logger.error("An error message: {}".format(err))

# set up an account access key 
spark.conf.set(
    "fs.azure.account.key.%s.blob.core.windows.net"%(storage_name),
    storage_key)

# transfer log file to azure blob storage
dbutils.fs.cp('file:%s'%(log_dir), 'wasbs://%s@%s.blob.core.windows.net//log/%s'%(container_name, storage_name, log_name))

2022-04-17 19:41:31.768174 Launching Chrome...
2022-04-17 19:41:32.469886 Chrome launched.
2022-04-17 19:41:33.794664 Browser ready to use.
2022-04-17 19:41:35.326394 downloading #1: https://store.cineplex.com/Product/spider-man-no-way-home .......
2022-04-17 19:41:41.337014 downloading #2: https://store.cineplex.com/Product/sing-2 .......
2022-04-17 19:41:47.206087 downloading #3: https://store.cineplex.com/Product/blacklight .......
2022-04-17 19:41:53.331906 downloading #4: https://store.cineplex.com/Product/the-matrix-resurrections .......
2022-04-17 19:41:59.193820 downloading #5: https://store.cineplex.com/Product/dog .......
2022-04-17 19:42:09,591 - cineplex_scraper - WARNING - 2175 - An info message: Message: 
Stacktrace:
#0 0x555e77d3dad3 <unknown>
#1 0x555e77a9d568 <unknown>
#2 0x555e77ad3c46 <unknown>
#3 0x555e77ad3e01 <unknown>
#4 0x555e77b06a64 <unknown>
#5 0x555e77af161d <unknown>
#6 0x555e77b04824 <unknown>
#7 0x555e77af14e3 <unknown>
#8 0x555e77ac6d1a <unknown>
#9 0x555e77ac7e75 <unknown>
#10 0x555e77d6befd <unknown>
#11 0x555e77d8519b <unknown>
#12 0x555e77d6dc65 <unknown>
#13 0x555e77d85ec8 <unknown>
#14 0x555e77d61360 <unknown>
#15 0x555e77da1a68 <unknown>
#16 0x555e77da1be8 <unknown>
#17 0x555e77dbb7fd <unknown>
#18 0x7fc1d239e609 <unknown>

2022-04-17 19:42:10.808895 downloading #6: https://store.cineplex.com/Product/marry-me .......
2022-04-17 19:42:21,192 - cineplex_scraper - WARNING - 2175 - An info message: Message: 
Stacktrace:
#0 0x555e77d3dad3 <unknown>
#1 0x555e77a9d568 <unknown>
#2 0x555e77ad3c46 <unknown>
#3 0x555e77ad3e01 <unknown>
#4 0x555e77b06a64 <unknown>
#5 0x555e77af161d <unknown>
#6 0x555e77b04824 <unknown>
#7 0x555e77af14e3 <unknown>
#8 0x555e77ac6d1a <unknown>
#9 0x555e77ac7e75 <unknown>
#10 0x555e77d6befd <unknown>
#11 0x555e77d8519b <unknown>
#12 0x555e77d6dc65 <unknown>
#13 0x555e77d85ec8 <unknown>
#14 0x555e77d61360 <unknown>
#15 0x555e77da1a68 <unknown>
#16 0x555e77da1be8 <unknown>
#17 0x555e77dbb7fd <unknown>
#18 0x7fc1d239e609 <unknown>

2022-04-17 19:42:22.100440 downloading #7: https://store.cineplex.com/Product/nightmare-alley .......
2022-04-17 19:42:28.195603 downloading #8: https://store.cineplex.com/Product/death-on-the-nile .......
2022-04-17 19:42:34.359690 downloading #9: https://store.cineplex.com/Product/ghostbusters-afterlife .......
2022-04-17 19:42:40.237646 downloading #10: https://store.cineplex.com/Product/scream-2022 .......
2022-04-17 19:42:46.349974 downloading #11: https://store.cineplex.com/Product/house-of-gucci .......
2022-04-17 19:42:53.320346 downloading #12: https://store.cineplex.com/Product/studio-666 .......
2022-04-17 19:43:03,701 - cineplex_scraper - WARNING - 2175 - An info message: Message: 
Stacktrace:
#0 0x555e77d3dad3 <unknown>
#1 0x555e77a9d568 <unknown>
#2 0x555e77ad3c46 <unknown>
#3 0x555e77ad3e01 <unknown>
#4 0x555e77b06a64 <unknown>
#5 0x555e77af161d <unknown>
#6 0x555e77b04824 <unknown>
#7 0x555e77af14e3 <unknown>
#8 0x555e77ac6d1a <unknown>
#9 0x555e77ac7e75 <unknown>
#10 0x555e77d6befd <unknown>
#11 0x555e77d8519b <unknown>
#12 0x555e77d6dc65 <unknown>
#13 0x555e77d85ec8 <unknown>
#14 0x555e77d61360 <unknown>
#15 0x555e77da1a68 <unknown>
#16 0x555e77da1be8 <unknown>
#17 0x555e77dbb7fd <unknown>
#18 0x7fc1d239e609 <unknown>

2022-04-17 19:43:04.774687 downloading #13: https://store.cineplex.com/Product/clifford-the-big-red-dog .......
2022-04-17 19:43:10.823134 downloading #14: https://store.cineplex.com/Product/no-time-to-die803354 .......
2022-04-17 19:43:17.010439 downloading #15: https://store.cineplex.com/Product/venom-let-there-be-carnage .......
2022-04-17 19:43:23.071848 downloading #16: https://store.cineplex.com/Product/licorice-pizza .......
2022-04-17 19:43:29.194456 downloading #17: https://store.cineplex.com/Product/spencer .......
2022-04-17 19:43:35.415917 downloading #18: https://store.cineplex.com/Product/resident-evil-welcome-to-raccoon-city .......
2022-04-17 19:43:41.416666 downloading #19: https://store.cineplex